In [16]:
import numpy as np
import pandas as pd

### 金融信贷特征衍生代码及字段含义

In [ ]:
def generate_pre_dws_group_cte():
    """生成预处理去重的CTE（核心优化：提前去重，避免重复distinct）"""
    pre_dws_group_sql = """
        WITH base_tag_layer AS (
        SELECT 
            mobile_md5,
            back_date,
            sign,
            event_type,
            ind_tag,
            count(mobile_md5) as times,
            concat_ws(',',collect_set(sign)) as sign_concat
        FROM(
            SELECT
                mobile_md5,
                back_date,
                sign,
                event_type,
                ind_tag,
                CASE 
                    WHEN DATEDIFF(back_date, event_time) > 0 AND DATEDIFF(back_date, event_time) <= 15 THEN 1
                    WHEN DATEDIFF(back_date, event_time) > 15 AND DATEDIFF(back_date, event_time) <= 30 THEN 2
                    WHEN DATEDIFF(back_date, event_time) > 30 AND DATEDIFF(back_date, event_time) <= 45 THEN 3
                    WHEN DATEDIFF(back_date, event_time) > 45 AND DATEDIFF(back_date, event_time) <= 60 THEN 4
                    WHEN DATEDIFF(back_date, event_time) > 60 AND DATEDIFF(back_date, event_time) <= 75 THEN 5
                    WHEN DATEDIFF(back_date, event_time) > 75 AND DATEDIFF(back_date, event_time) <= 90 THEN 6
                    WHEN DATEDIFF(back_date, event_time) > 90 AND DATEDIFF(back_date, event_time) <= 105 THEN 7
                    WHEN DATEDIFF(back_date, event_time) > 105 AND DATEDIFF(back_date, event_time) <= 120 THEN 8
                    WHEN DATEDIFF(back_date, event_time) > 120 AND DATEDIFF(back_date, event_time) <= 135 THEN 9
                    WHEN DATEDIFF(back_date, event_time) > 135 AND DATEDIFF(back_date, event_time) <= 150 THEN 10
                    WHEN DATEDIFF(back_date, event_time) > 150 AND DATEDIFF(back_date, event_time) <= 165 THEN 11
                    WHEN DATEDIFF(back_date, event_time) > 165 AND DATEDIFF(back_date, event_time) <= 180 THEN 12
                    WHEN DATEDIFF(back_date, event_time) > 180 AND DATEDIFF(back_date, event_time) <= 195 THEN 13
                    WHEN DATEDIFF(back_date, event_time) > 195 AND DATEDIFF(back_date, event_time) <= 210 THEN 14
                    WHEN DATEDIFF(back_date, event_time) > 210 AND DATEDIFF(back_date, event_time) <= 225 THEN 15
                    WHEN DATEDIFF(back_date, event_time) > 225 AND DATEDIFF(back_date, event_time) <= 240 THEN 16
                    WHEN DATEDIFF(back_date, event_time) > 240 AND DATEDIFF(back_date, event_time) <= 255 THEN 17
                    WHEN DATEDIFF(back_date, event_time) > 255 AND DATEDIFF(back_date, event_time) <= 270 THEN 18
                    WHEN DATEDIFF(back_date, event_time) > 270 AND DATEDIFF(back_date, event_time) <= 285 THEN 19
                    WHEN DATEDIFF(back_date, event_time) > 285 AND DATEDIFF(back_date, event_time) <= 300 THEN 20
                    WHEN DATEDIFF(back_date, event_time) > 300 AND DATEDIFF(back_date, event_time) <= 315 THEN 21
                    WHEN DATEDIFF(back_date, event_time) > 315 AND DATEDIFF(back_date, event_time) <= 330 THEN 22
                    WHEN DATEDIFF(back_date, event_time) > 330 AND DATEDIFF(back_date, event_time) <= 345 THEN 23
                    ELSE 24 
                END AS dt_tag
            FROM customer_dwb_sms
            WHERE sign IS NOT NULL
            )customer_dwb_sms 
        GROUP BY mobile_md5, back_date, sign, event_type, ind_tag, dt_tag
        )"""
    return pre_dws_group_sql

def generate_plat_cnt_fields(win_days, ind_arr, event_arr, field_meta):
    """
    生成集中计算count(distinct)的平台数字段（只扫描一次数据）
    :return: 平台数统计字段字符串、平台数CTE名称
    """
    plat_cnt_fields = []
    # 行业中文注释映射
    ind_cn_mapping = {
        "indA": "A类小贷",
        "indB": "B类小贷",
        "indC": "C类小贷",
        "indD": "三方催收",
        "indE": "消费金融",
        "indF": "银行",
        "indG": "其他机构"
    }
    # 事件中文注释映射
    event_cn_mapping = {
        "B01": "严重逾期",
        "B02": "三方催收",
        "B03": "中度逾期",
        "B04": "还款失败",
        "B05": "申请失败",
        "B06": "还款成功",
        "B07": "放款成功",
        "B08": "还款提醒",
        "B09": "贷款营销",
        "B10": "金融其他"
    }

    # ========== 1. 总平台数 ==========
    for win in win_days:
        flag = f"flag_{win}d"
        plat_cnt_field = f"sp_fin_loan_plat_cnt_{win}d"
        plat_cnt_fields.append(f"""
            COUNT(DISTINCT CASE WHEN {flag} = 1 THEN sign END) AS {plat_cnt_field}""")
        field_meta[plat_cnt_field] = {
            "type": "INT",
            "cn_desc": f"近{win}天内金融事件平台数",
            "biz_tag": "平台数"
        }

    # # ========== 2. 行业平台数 ==========
    # for win in win_days:
    #     flag = f"flag_{win}d"
    #     for ind in ind_arr:
    #         ind_lower = ind.lower()
    #         ind_cn = ind_cn_mapping.get(ind, ind)
    #         ind_plat_field = f"sp_fin_loan_{ind_lower}_plat_cnt_{win}d"
    #         plat_cnt_fields.append(f"""
    #         COUNT(DISTINCT CASE WHEN {flag} = 1 AND ind_tag='{ind}' THEN sign END) AS {ind_plat_field}""")
    #         field_meta[ind_plat_field] = {
    #             "type": "INT",
    #             "cn_desc": f"近{win}天内{ind_cn}行业金融事件平台数",
    #             "biz_tag": f"行业平台数|{ind_cn}"
    #         }

    # ========== 3. 事件平台数 ==========
    for win in win_days:
        flag = f"flag_{win}d"
        for event in event_arr:
            event_lower = event.lower()
            event_cn = event_cn_mapping.get(event, event)
            event_plat_field = f"sp_fin_loan_event_{event_lower}_plat_cnt_{win}d"
            plat_cnt_fields.append(f"""
            COUNT(DISTINCT CASE WHEN {flag} = 1 AND event_type='{event}' THEN sign END) AS {event_plat_field}""")
            field_meta[event_plat_field] = {
                "type": "INT",
                "cn_desc": f"近{win}天内{event_cn}事件平台数",
                "biz_tag": f"事件平台数|{event_cn}"
            }

    # ========== 4. 行业+事件组合平台数 ==========
    for win in win_days:
        flag = f"flag_{win}d"
        for ind in ind_arr:
            ind_lower = ind.lower()
            if ind_lower == "indd":
                continue
            for event in event_arr:
                event_lower = event.lower()
                event_cn = event_cn_mapping.get(event, event)
                ind_cn = ind_cn_mapping.get(ind, ind)
                combo_plat_field = f"sp_fin_loan_{ind_lower}_event_{event_lower}_plat_cnt_{win}d"
                plat_cnt_fields.append(f"""
                COUNT(DISTINCT CASE WHEN {flag} = 1 AND ind_tag='{ind}' AND event_type='{event}' THEN sign END) AS {combo_plat_field}""")
                field_meta[combo_plat_field] = {
                    "type": "INT",
                    "cn_desc": f"近{win}天内{ind_cn}行业{event_cn}事件平台数",
                    "biz_tag": f"行业+事件平台数|{ind_cn}|{event_cn}"
                }

    # 拼接字段，生成平台数统计CTE
    plat_cnt_fields_sql = ',\n'.join([f.strip() for f in plat_cnt_fields])
    plat_cnt_sql = f""",plat_cnt_stats AS (
        SELECT
            mobile_md5,
            back_date,
            {plat_cnt_fields_sql}
        FROM pre_distinct
        GROUP BY mobile_md5, back_date
    )"""
    return plat_cnt_sql, "plat_cnt_stats"

def generate_event_stats_fields(win_days, ind_arr, event_arr, field_meta):
    """
    生成轻量聚合字段（sum/MAX/MIN），不包含count(distinct)
    """
    fields = []
    # 行业中文注释映射
    ind_cn_mapping = {
        "indA": "A类小贷",
        "indB": "B类小贷",
        "indC": "C类小贷",
        "indD": "三方催收",
        "indE": "消费金融",
        "indF": "银行",
        "indG": "其他机构"
    }
    # 事件中文注释映射
    event_cn_mapping = {
        "B01": "严重逾期",
        "B02": "三方催收",
        "B03": "中度逾期",
        "B04": "还款失败",
        "B05": "申请失败",
        "B06": "还款成功",
        "B07": "放款成功",
        "B08": "还款提醒",
        "B09": "贷款营销",
        "B10": "金融其他"
    }

    # ========== 1. 金融事件总次数 ==========
    for win in win_days:
        flag = f"flag_{win}d"
        # 总次数字段
        total_times_field = f"sp_fin_loan_total_times_{win}d"
        fields.append(f"""
            SUM({flag}) AS {total_times_field}""")
        field_meta[total_times_field] = {
            "type": "BIGINT",
            "cn_desc": f"近{win}天内金融事件总次数",
            "biz_tag": "总次数"
        }

    # ========== 2. 不同行业行为次数 ==========
    for win in win_days:
        flag = f"flag_{win}d"
        for ind in ind_arr:
            ind_lower = ind.lower()
            ind_cn = ind_cn_mapping.get(ind, ind)
            # 行业次数字段
            ind_times_field = f"sp_fin_loan_{ind_lower}_times_{win}d"
            fields.append(f"""
            SUM(CASE WHEN {flag} = 1 AND ind_tag='{ind}' THEN 1 ELSE 0 END) AS {ind_times_field}""")
            field_meta[ind_times_field] = {
                "type": "BIGINT",
                "cn_desc": f"近{win}天内{ind_cn}行业金融事件次数",
                "biz_tag": f"行业次数|{ind_cn}"
            }

    # ========== 3. 不同事件类型行为次数 ==========
    for win in win_days:
        flag = f"flag_{win}d"
        for event in event_arr:
            event_lower = event.lower()
            event_cn = event_cn_mapping.get(event, event)
            # 事件次数字段
            event_times_field = f"sp_fin_loan_event_{event_lower}_times_{win}d"
            fields.append(f"""
            SUM(CASE WHEN {flag} = 1 AND event_type='{event}' THEN 1 ELSE 0 END) AS {event_times_field}""")
            field_meta[event_times_field] = {
                "type": "BIGINT",
                "cn_desc": f"近{win}天内{event_cn}事件次数",
                "biz_tag": f"事件次数|{event_cn}"
            }

    # ========== 4. 行业+事件类型组合次数 ==========
    for win in win_days:
        flag = f"flag_{win}d"
        for ind in ind_arr:
            ind_lower = ind.lower()
            if ind_lower == "indd":
                continue
            ind_cn = ind_cn_mapping.get(ind, ind)
            for event in event_arr:
                event_lower = event.lower()
                event_cn = event_cn_mapping.get(event, event)
                # 组合次数字段
                combo_times_field = f"sp_fin_loan_{ind_lower}_event_{event_lower}_times_{win}d"
                fields.append(f"""
                SUM(CASE WHEN {flag} = 1 AND ind_tag='{ind}' AND event_type='{event}' THEN 1 ELSE 0 END) AS {combo_times_field}""")
                field_meta[combo_times_field] = {
                    "type": "BIGINT",
                    "cn_desc": f"近{win}天内{ind_cn}行业{event_cn}事件次数",
                    "biz_tag": f"行业+事件次数|{ind_cn}|{event_cn}"
                }

    # ========== 5. 不同事件行为最近/最远一次距今天数 ==========
    flag = f"flag_{win}d"
    # 单个事件时间差
    for event in event_arr:
        event_lower = event.lower()
        event_cn = event_cn_mapping.get(event, event)
        # 最近天数
        event_latest_field = f"sp_fin_loan_event_{event_lower}_recent_dura_d_beforep_{win}d"
        fields.append(f"""
        datediff(back_date, MAX(CASE WHEN {flag} = 1 AND event_type='{event}' THEN event_time END)) AS {event_latest_field}""")
        field_meta[event_latest_field] = {
            "type": "INT",
            "cn_desc": f"近{win}天内{event_cn}事件最近一次距今天数",
            "biz_tag": f"事件天数|{event_cn}|最近"
        }
        # 最远天数
        event_farthest_field = f"sp_fin_loan_event_{event_lower}_remote_dura_d_beforep_{win}d"
        fields.append(f"""
        datediff(back_date, MIN(CASE WHEN {flag} = 1 AND event_type='{event}' THEN event_time END)) AS {event_farthest_field}""")
        field_meta[event_farthest_field] = {
            "type": "INT",
            "cn_desc": f"近{win}天内{event_cn}事件最远一次距今天数",
            "biz_tag": f"事件天数|{event_cn}|最远"
        }
    
    # 行业+事件组合时间差
    for ind in ind_arr:
        ind_lower = ind.lower()
        if ind_lower == "indd":
            continue
        ind_cn = ind_cn_mapping.get(ind, ind)
        for event in event_arr:
            event_lower = event.lower()
            event_cn = event_cn_mapping.get(event, event)
            # 最近天数
            combo_latest_field = f"sp_fin_loan_{ind_lower}_event_{event_lower}_recent_dura_d_beforep_{win}d"
            fields.append(f"""
            datediff(back_date, MAX(CASE WHEN {flag} = 1 AND ind_tag='{ind}' AND event_type='{event}' THEN event_time END)) AS {combo_latest_field}""")
            field_meta[combo_latest_field] = {
                "type": "INT",
                "cn_desc": f"近{win}天内{ind_cn}行业{event_cn}事件最近一次距今天数",
                "biz_tag": f"行业+事件天数|{ind_cn}|{event_cn}|最近"
            }
            # 最远天数
            combo_farthest_field = f"sp_fin_loan_{ind_lower}_event_{event_lower}_remote_dura_d_beforep_{win}d"
            fields.append(f"""
            datediff(back_date, MIN(CASE WHEN {flag} = 1 AND ind_tag='{ind}' AND event_type='{event}' THEN event_time END)) AS {combo_farthest_field}""")
            field_meta[combo_farthest_field] = {
                "type": "INT",
                "cn_desc": f"近{win}天内{ind_cn}行业{event_cn}事件最远一次距今天数",
                "biz_tag": f"行业+事件天数|{ind_cn}|{event_cn}|最远"
            }

    return ",\n".join([f.strip() for f in fields])

def generate_fin_loan_stat_fields_levelv2(win_days, event_arr, ind_arr, field_meta):
    """
    生成比率类字段（逻辑不变，适配新的字段结构）
    """
    fields = []
    # 事件/行业中文映射（复用）
    event_cn_mapping = {
        "B01": "严重逾期",
        "B02": "三方催收",
        "B03": "中度逾期",
        "B04": "还款失败",
        "B05": "申请失败",
        "B06": "还款成功",
        "B07": "放款成功",
        "B08": "还款提醒",
        "B09": "贷款营销",
        "B10": "金融其他"
    }
    ind_cn_mapping = {
        "indA": "A类小贷",
        "indB": "B类小贷",
        "indC": "C类小贷",
        "indD": "三方催收",
        "indE": "消费金融",
        "indF": "银行",
        "indG": "其他机构"
    }
    
    # ========== 6. 不同事件/行业占总事件比率 ==========
    for win in win_days:
        # 6.1 单个事件占总事件的比率
        for event in event_arr:
            event_lower = event.lower()
            event_cn = event_cn_mapping.get(event, event)
            event_ratio_field = f"sp_fin_loan_event_{event_lower}_ratio_total_{win}d"
            fields.append(f"""
            ROUND(CASE WHEN sp_fin_loan_total_times_{win}d = 0 THEN 0 ELSE sp_fin_loan_event_{event_lower}_times_{win}d / sp_fin_loan_total_times_{win}d END, 6) AS {event_ratio_field}""")
            field_meta[event_ratio_field] = {
                "type": "DOUBLE",
                "cn_desc": f"近{win}天内{event_cn}事件次数占总事件次数的比率（保留6位小数）",
                "biz_tag": f"事件占比|{event_cn}"
            }
        
        # 6.2 单个行业占总事件的比率
        for ind in ind_arr:
            ind_lower = ind.lower()
            ind_cn = ind_cn_mapping.get(ind, ind)
            ind_ratio_field = f"sp_fin_loan_{ind_lower}_ratio_total_{win}d"
            fields.append(f"""
            ROUND(CASE WHEN sp_fin_loan_total_times_{win}d = 0 THEN 0 ELSE sp_fin_loan_{ind_lower}_times_{win}d / sp_fin_loan_total_times_{win}d END, 6) AS {ind_ratio_field}""")
            field_meta[ind_ratio_field] = {
                "type": "DOUBLE",
                "cn_desc": f"近{win}天内{ind_cn}行业事件次数占总事件次数的比率（保留6位小数）",
                "biz_tag": f"行业占比|{ind_cn}"
            }

    # ========== 7. 不同事件/行业+事件跨窗口占比 ==========
    win_mapping = {
        15: 30,   # 15天/30天
        30: 90,   # 30天/90天
        # 60: 360   # 60天/360天
    }
    for small_win in [15, 30]:
        big_win = win_mapping[small_win]
        
        # 7.1 纯事件维度跨窗口占比
        for event in event_arr:
            event_lower = event.lower()
            event_cn = event_cn_mapping.get(event, event)
            event_win_ratio_field = f"sp_fin_loan_event_{event_lower}_ratio_{small_win}to{big_win}d"
            fields.append(f"""ROUND(CASE WHEN sp_fin_loan_event_{event_lower}_times_{big_win}d = 0 THEN 0 ELSE sp_fin_loan_event_{event_lower}_times_{small_win}d / sp_fin_loan_event_{event_lower}_times_{big_win}d END, 6) AS {event_win_ratio_field}""")
            field_meta[event_win_ratio_field] = {
                "type": "DOUBLE",
                "cn_desc": f"近{small_win}天内{event_cn}事件次数占近{big_win}天次数的比率（保留6位小数）",
                "biz_tag": f"事件跨窗口占比|{event_cn}|{small_win}d/{big_win}d"
            }
        
        # 7.2 行业+事件组合维度跨窗口占比
        for ind in ind_arr:
            ind_lower = ind.lower()
            if ind_lower == "indd":
                continue
            ind_cn = ind_cn_mapping.get(ind, ind)
            for event in event_arr:
                event_lower = event.lower()
                event_cn = event_cn_mapping.get(event, event)
                combo_win_ratio_field = f"sp_fin_loan_{ind_lower}_event_{event_lower}_ratio_{small_win}to{big_win}d"
                fields.append(f"""ROUND(CASE WHEN sp_fin_loan_{ind_lower}_event_{event_lower}_times_{big_win}d = 0 THEN 0 ELSE sp_fin_loan_{ind_lower}_event_{event_lower}_times_{small_win}d / sp_fin_loan_{ind_lower}_event_{event_lower}_times_{big_win}d END, 6) AS {combo_win_ratio_field}""")
                field_meta[combo_win_ratio_field] = {
                    "type": "DOUBLE",
                    "cn_desc": f"近{small_win}天内{ind_cn}行业{event_cn}事件次数占近{big_win}天次数的比率（保留6位小数）",
                    "biz_tag": f"行业+事件跨窗口占比|{ind_cn}|{event_cn}|{small_win}d/{big_win}d"
                }
    
    return ",\n".join([f.strip() for f in fields])

# ===================== 主流程 =====================
# 初始化字段元数据字典
field_mapping_dict = {
    # 基础字段（非统计字段）
    "mobile_md5": {"type": "STRING", "cn_desc": "手机号MD5加密值", "biz_tag": "基础标识"},
    "back_date": {"type": "DATE", "cn_desc": "回溯日期", "biz_tag": "基础标识"}
}

# 配置项
WIN_DAYS = [15, 30, 90, 180, 360]
IND_ARR = ["indA", "indB", "indC", "indD", "indE", "indF", "indG"]
EVENT_ARR = ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B09", "B10"]

# 基础SQL（替换为实际值）
base_sql = """
CREATE TABLE sms_bd_data.customer_test_offline_fin_loan_feature_2026011001 AS
WITH cus_sample AS (
    SELECT 
        mobile_md5,
        back_date
    FROM(
        -- 需要满足回溯日期
        -- back_date要求YYYYMMDD格式
        select
            phone as mobile_md5,
            substr(submitdate, 0, 10) as back_date
        from sms_bd_data.ods_yxx_spn_sms_detail_di
        where submitdate>='2025-08-20'
        limit 50000
    )smp
    group by mobile_md5, back_date
),
customer_dwb_sms AS (
    select
        mobile_md5, 
        back_date, 
        sign, 
        event_time,
        event_type, 
        ind_tag
    from(
        select
            cus_smp.mobile_md5,
            cus_smp.back_date,
            dwb_loan.sign,
            dwb_loan.event_type,
            dwb_loan.event_time,
            case when dwb_loan.sign regexp "抖音月付|放心借|微众银行|花呗" then 'indA'
                 when dwb_loan.sign regexp "美团月付|携程金融|360借|拍拍贷|分期乐|拍拍金融|360贷款|上海拍拍贷|美团金融服务" then 'indB'
                 when dwb_loan.sign regexp "还呗|信用飞钱包|省呗|乐享借|融360|洋钱罐借款|众安金融|分期金融|信用飞|乐逸花|极融借款|好分期|信用卡贷|金瀛分期|小赢卡贷|宜享花|众安贷借钱|融逸花|卡贷金融|吉用花|时光分期|小花钱包|借钱呗|金豆花|信用飞金融|来分期|小花借款|好会借|乐贷分期|及贷|榕树贷款|融易分期|借小花|薇钱包|你我贷|玖富借条" then 'indC'
                 when dwb_loan.sign regexp "金融调解|数科金融|满松科技|利信普惠|普信金融|卡卡金融|和信普惠|和信金融|利信金融|数信普惠|普惠快信|普惠信息|首山金融|普惠金融|数信普惠|玖富万卡|钱站|鹰潭市金融纠纷调解中心|国美易卡|普惠分期|上海金融|数科纠纷调解中心" or sign in ('玖富') then 'indD'
                 when dwb_loan.sign regexp "消费分期|消费金融|招联金融|移动白条|捷信金融|马上金融|中原消费金融|马上消费|分期消费" then 'indE'
                 when dwb_loan.sign regexp "银行|农信|农商" then 'indF'
            else 'indG' end as ind_tag
        from cus_sample cus_smp
        left join(
            select
                phone,
                sign,
                event_type,
                from_unixtime(unix_timestamp(the_date, 'yyyyMMdd'), 'yyyy-MM-dd') as event_time
            from sms_bd_data.sms_dwb_fin_loan_event_fdt
        )dwb_loan
        on cus_smp.mobile_md5=dwb_loan.phone 
        where cus_smp.back_date > dwb_loan.event_time and dwb_loan.event_time >= date_sub(cus_smp.back_date, 360)
    )cus_dwb
    group by mobile_md5, back_date, sign, event_type, event_time, ind_tag
)
"""

# 生成预处理去重CTE
pre_distinct_sql = generate_pre_distinct_cte()

# 生成平台数统计字段（集中计算count(distinct)）
plat_cnt_sql, plat_cnt_cte_name = generate_plat_cnt_fields(WIN_DAYS, IND_ARR, EVENT_ARR, field_mapping_dict)

# 生成轻量聚合字段（sum/MAX/MIN）
event_stats_fields = generate_event_stats_fields(WIN_DAYS, IND_ARR, EVENT_ARR, field_mapping_dict)

# 生成事件统计CTE
event_stats_sql = f""",event_stats AS (
    SELECT
        mobile_md5,
        back_date,
        {event_stats_fields}
    FROM pre_distinct
    GROUP BY mobile_md5, back_date
)
"""

# 生成比率类字段
ratio_fields = generate_fin_loan_stat_fields_levelv2(WIN_DAYS, EVENT_ARR, IND_ARR, field_mapping_dict)

# 拼接最终SQL
final_sql = f"""{base_sql}
{pre_distinct_sql}
{plat_cnt_sql}
{event_stats_sql},aggregated_data AS (
    SELECT
        es.*,
        pcs.*
    FROM event_stats es
    LEFT JOIN {plat_cnt_cte_name} pcs
    ON es.mobile_md5 = pcs.mobile_md5 AND es.back_date = pcs.back_date
)
SELECT
   *,
    {ratio_fields}
FROM aggregated_data
"""

# ===================== 输出结果 =====================
# 打印生成的SQL（截取前3000字符，避免过长）
print(final_sql)



CREATE TABLE sms_bd_data.customer_test_offline_fin_loan_feature_2026011001 AS
WITH cus_sample AS (
    SELECT 
        mobile_md5,
        back_date
    FROM(
        -- 需要满足回溯日期
        -- back_date要求YYYYMMDD格式
        select
            phone as mobile_md5,
            substr(submitdate, 0, 10) as back_date
        from sms_bd_data.ods_yxx_spn_sms_detail_di
        where submitdate>='2025-08-20'
        limit 50000
    )smp
    group by mobile_md5, back_date
),
customer_dwb_sms AS (
    select
        mobile_md5, 
        back_date, 
        sign, 
        event_time,
        event_type, 
        ind_tag
    from(
        select
            cus_smp.mobile_md5,
            cus_smp.back_date,
            dwb_loan.sign,
            dwb_loan.event_type,
            dwb_loan.event_time,
            case when dwb_loan.sign regexp "抖音月付|放心借|微众银行|花呗" then 'indA'
                 when dwb_loan.sign regexp "美团月付|携程金融|360借|拍拍贷|分期乐|拍拍金融|360贷款|上海拍拍贷|美团金融服务" then 'indB'
                 when dwb_

In [18]:
# 3. 输出字段类型统计
print("\n===== Express模块字段类型统计 =====")
type_count = {}
for field, meta in field_mapping_dict.items():
    field_type = meta["type"]
    type_count[field_type] = type_count.get(field_type, 0) + 1
for t, cnt in type_count.items():
    print(f"{t}类型字段数：{cnt}")


===== Express模块字段类型统计 =====
STRING类型字段数：1
DATE类型字段数：1
INT类型字段数：608
BIGINT类型字段数：468
DOUBLE类型字段数：312


In [19]:
field_mapping_dataframe = pd.DataFrame(field_mapping_dict).T
field_mapping_dataframe.to_excel("field_mapping_output_v2.xlsx", index_label="field_name")

### 快递类特征衍生代码及字段含义

In [ ]:
def generate_pre_distinct_cte():
    """生成预处理去重的CTE（核心优化：提前去重，避免重复distinct）"""
    pre_distinct_sql = """,pre_distinct AS (
    SELECT
        mobile_md5,
        back_date,
        sign,
        event_type,
        event_time,
        CASE WHEN datediff(back_date, event_time) > 0 AND datediff(back_date, event_time) <= 15 THEN 1 ELSE 0 END as flag_15d,
        CASE WHEN datediff(back_date, event_time) > 0 AND datediff(back_date, event_time) <= 30 THEN 1 ELSE 0 END as flag_30d,
        CASE WHEN datediff(back_date, event_time) > 0 AND datediff(back_date, event_time) <= 90 THEN 1 ELSE 0 END as flag_90d,
        CASE WHEN datediff(back_date, event_time) > 0 AND datediff(back_date, event_time) <= 180 THEN 1 ELSE 0 END as flag_180d,
        CASE WHEN datediff(back_date, event_time) > 0 AND datediff(back_date, event_time) <= 360 THEN 1 ELSE 0 END as flag_360d
    FROM customer_dwb_express
    WHERE sign IS NOT NULL
    )
    """
    return pre_distinct_sql

def generate_plat_cnt_fields(win_days, event_arr, field_meta):
    """用 SUM+GROUP BY 替代 COUNT(DISTINCT)，效率最高"""
    # 第一步：预去重中间表（每个 sign 在 (mobile_md5, back_date, 窗口) 下只留1条）
    pre_distinct_sql = """,distinct_plat AS (
        SELECT
            mobile_md5,
            back_date,
            sign,
            event_type,
            flag_15d, flag_30d, flag_90d, flag_180d, flag_360d
        FROM pre_distinct
        GROUP BY mobile_md5, back_date, sign, event_type, flag_15d, flag_30d, flag_90d, flag_180d, flag_360d
    )"""

    # 第二步：用 SUM(CASE) 统计去重后的数量（替代 COUNT(DISTINCT)）
    plat_cnt_fields = []
    event_cn_mapping = {"A01":"快递签收","A02":"快递投诉","A03":"快递寄件","A04":"其他快递"}
    
    # 总平台数
    for win in win_days:
        flag = f"flag_{win}d"
        field = f"sp_express_plat_cnt_{win}d"
        plat_cnt_fields.append(f"SUM(CASE WHEN {flag}=1 THEN 1 ELSE 0 END) AS {field}")
        field_meta[field] = {"type":"INT", "cn_desc":f"近{win}天快递平台数", "biz_tag":"平台数"}
    
    # 事件维度平台数
    for win in win_days:
        flag = f"flag_{win}d"
        for event in event_arr:
            event_lower = event.lower()
            field = f"sp_express_event_{event_lower}_plat_cnt_{win}d"
            plat_cnt_fields.append(f"SUM(CASE WHEN {flag}=1 AND event_type='{event}' THEN 1 ELSE 0 END) AS {field}")
            field_meta[field] = {"type":"INT", "cn_desc":f"近{win}天{event_cn_mapping[event]}平台数", "biz_tag":f"事件平台数|{event}"}

    # 拼接最终CTE
    plat_cnt_sql = f"""{pre_distinct_sql},plat_cnt_stats AS (
        SELECT
            mobile_md5,
            back_date,
            {',\n'.join(plat_cnt_fields)}
        FROM distinct_plat
        GROUP BY mobile_md5, back_date
    )"""
    return plat_cnt_sql, "plat_cnt_stats"
def generate_event_stats_fields(win_days, event_arr, field_meta):
    """
    生成轻量聚合字段（sum/MAX/MIN），不包含count(distinct)
    """
    fields = []
    # 快递事件中文注释映射
    event_cn_mapping = {
        "A01": "快递签收",
        "A02": "快递投诉",
        "A03": "快递寄件",
        "A04": "其他快递"
    }

    # ========== 1. 快递事件总次数 ==========
    for win in win_days:
        flag = f"flag_{win}d"
        # 总次数字段
        total_times_field = f"sp_express_total_times_{win}d"
        fields.append(f"""SUM({flag}) AS {total_times_field}""")
        field_meta[total_times_field] = {
            "type": "BIGINT",
            "cn_desc": f"近{win}天内快递事件总次数",
            "biz_tag": "总次数"
        }

    # ========== 2. 不同事件类型行为次数 ==========
    for win in win_days:
        flag = f"flag_{win}d"
        for event in event_arr:
            event_lower = event.lower()
            event_cn = event_cn_mapping.get(event, event)
            # 事件次数字段
            event_times_field = f"sp_express_event_{event_lower}_times_{win}d"
            fields.append(f"""
            SUM(CASE WHEN {flag} = 1 AND event_type='{event}' THEN 1 ELSE 0 END) AS {event_times_field}""")
            field_meta[event_times_field] = {
                "type": "BIGINT",
                "cn_desc": f"近{win}天内{event_cn}事件次数",
                "biz_tag": f"事件次数|{event_cn}"
            }

    # ========== 3. 不同事件行为最近/最远一次距今天数 ==========
    flag = f"flag_{win}d"
    # 单个事件时间差
    for event in event_arr:
        event_lower = event.lower()
        event_cn = event_cn_mapping.get(event, event)
        # 最近天数
        event_latest_field = f"sp_express_event_{event_lower}_recent_dura_d_beforep_{win}d"
        fields.append(f"""
        datediff(back_date, MAX(CASE WHEN {flag} = 1 AND event_type='{event}' THEN event_time END)) AS {event_latest_field}""")
        field_meta[event_latest_field] = {
            "type": "INT",
            "cn_desc": f"近{win}天内{event_cn}事件最近一次距今天数",
            "biz_tag": f"事件天数|{event_cn}|最近"
        }
        # 最远天数
        event_farthest_field = f"sp_express_event_{event_lower}_remote_dura_d_beforep_{win}d"
        fields.append(f"""
        datediff(back_date, MIN(CASE WHEN {flag} = 1 AND event_type='{event}' THEN event_time END)) AS {event_farthest_field}""")
        field_meta[event_farthest_field] = {
            "type": "INT",
            "cn_desc": f"近{win}天内{event_cn}事件最远一次距今天数",
            "biz_tag": f"事件天数|{event_cn}|最远"
        }

    # 拼接字段，去除多余换行和空格
    return ",\n".join([f.strip() for f in fields])

def generate_express_stat_fields_levelv2(win_days, event_arr, field_meta):
    """
    生成快递模块比率类字段，完全对齐fin_loan逻辑
    """
    fields = []
    # 快递事件中文注释映射（复用）
    event_cn_mapping = {
        "A01": "快递签收",
        "A02": "快递投诉",
        "A03": "快递寄件",
        "A04": "其他快递"
    }
    
    # ========== 4. 不同事件占总事件的比率 ==========
    for win in win_days:
        # 单个事件占总事件的比率
        for event in event_arr:
            event_lower = event.lower()
            event_cn = event_cn_mapping.get(event, event)
            event_ratio_field = f"sp_express_event_{event_lower}_ratio_total_{win}d"
            fields.append(f"""
            ROUND(CASE WHEN sp_express_total_times_{win}d = 0 THEN 0 ELSE sp_express_event_{event_lower}_times_{win}d / sp_express_total_times_{win}d END, 6) AS {event_ratio_field}""")
            field_meta[event_ratio_field] = {
                "type": "DOUBLE",
                "cn_desc": f"近{win}天内{event_cn}事件次数占总事件次数的比率（保留6位小数）",
                "biz_tag": f"事件占比|{event_cn}"
            }

    # ========== 5. 不同事件跨窗口占比 ==========
    win_mapping = {
        15: 30,   # 15天/30天
        30: 90,   # 30天/90天
        90: 180,  # 90天/180天
        180: 360, # 180天/360天
        360: 360  # 360天自身
    }
    for small_win in [15, 30, 90]:
        big_win = win_mapping[small_win]
        # 仅处理存在的大窗口
        if big_win not in win_days:
            continue
        
        # 纯事件维度跨窗口占比
        for event in event_arr:
            event_lower = event.lower()
            event_cn = event_cn_mapping.get(event, event)
            event_win_ratio_field = f"sp_express_event_{event_lower}_ratio_{small_win}to{big_win}d"
            fields.append(f"""ROUND(CASE WHEN sp_express_event_{event_lower}_times_{big_win}d = 0 THEN 0 ELSE sp_express_event_{event_lower}_times_{small_win}d / sp_express_event_{event_lower}_times_{big_win}d END, 6) AS {event_win_ratio_field}""")
            field_meta[event_win_ratio_field] = {
                "type": "DOUBLE",
                "cn_desc": f"近{small_win}天内{event_cn}事件次数占近{big_win}天次数的比率（保留6位小数）",
                "biz_tag": f"事件跨窗口占比|{event_cn}|{small_win}d/{big_win}d"
            }
    
    return ",\n".join([f.strip() for f in fields])

# ===================== 主流程（完全对齐fin_loan） =====================
# 初始化字段元数据字典
exp_field_mapping_dict = {
    # 基础字段（非统计字段）
    "mobile_md5": {"type": "STRING", "cn_desc": "手机号MD5加密值", "biz_tag": "基础标识"},
    "back_date": {"type": "DATE", "cn_desc": "回溯日期", "biz_tag": "基础标识"}
}

# express模块配置项
WIN_DAYS = [15, 30, 90, 180, 360]
EVENT_ARR = ["A01", "A02", "A03", "A04"]

# 基础SQL（适配快递业务，结构对齐fin_loan）
base_sql = """
CREATE TABLE sms_bd_data.customer_test_offline_express_feature_2026010901 AS
WITH cus_sample AS (
    SELECT 
        mobile_md5,
        back_date
    FROM(
        -- 快递业务样本表
        select 
            phone as mobile_md5,
            substr(submitdate, 0, 10) as back_date
        from sms_bd_data.ods_yxx_spn_sms_detail_di
        where submitdate>='2025-08-20'
        limit 50000
    )smp
    group by mobile_md5, back_date
),
customer_dwb_express AS (
    select
        mobile_md5, 
        back_date, 
        sign, 
        event_type,
        event_time
    from(
        select
            cus_smp.mobile_md5,
            cus_smp.back_date,
            dwb_express.sign,
            dwb_express.event_type,
            dwb_express.event_time
        from cus_sample cus_smp
        left join(
            select
                phone,
                sign,
                event_type,
                from_unixtime(unix_timestamp(the_date, 'yyyyMMdd'), 'yyyy-MM-dd') as event_time
            from sms_bd_data.sms_dwb_express_event_fdt
        )dwb_express
        on cus_smp.mobile_md5=dwb_express.phone 
        where cus_smp.back_date > dwb_express.event_time and dwb_express.event_time >= date_sub(cus_smp.back_date, 360)
    )cus_dwb
    group by mobile_md5, back_date, sign, event_type, event_time
)
"""

# 生成预处理去重CTE（核心对齐点1）
pre_distinct_sql = generate_pre_distinct_cte()

# 生成平台数统计字段（集中计算count(distinct)，核心对齐点2）
plat_cnt_sql, plat_cnt_cte_name = generate_plat_cnt_fields(WIN_DAYS, EVENT_ARR, exp_field_mapping_dict)

# 生成轻量聚合字段（sum/MAX/MIN，核心对齐点3）
event_stats_fields = generate_event_stats_fields(WIN_DAYS, EVENT_ARR, exp_field_mapping_dict)

# 生成事件统计CTE（核心对齐点4）
event_stats_sql = f""",event_stats AS (
    SELECT
        mobile_md5,
        back_date,
        {event_stats_fields}
    FROM pre_distinct
    GROUP BY mobile_md5, back_date
)
"""

# 生成比率类字段
ratio_fields = generate_express_stat_fields_levelv2(WIN_DAYS, EVENT_ARR, exp_field_mapping_dict)

# 拼接最终SQL（采用fin_loan的兼容关联写法，核心对齐点5）
exp_final_sql = f"""{base_sql}
{pre_distinct_sql}
{plat_cnt_sql}
{event_stats_sql},aggregated_data AS (
    SELECT
        es.*,
        pcs.*
    FROM event_stats es
    LEFT JOIN {plat_cnt_cte_name} pcs
    ON es.mobile_md5 = pcs.mobile_md5 AND es.back_date = pcs.back_date
)
SELECT
   *,
    {ratio_fields}
FROM aggregated_data
"""

# ===================== 输出结果 =====================
# 打印生成的SQL（可直接复制到Hue执行）
print(exp_final_sql)



CREATE TABLE sms_bd_data.customer_test_offline_express_feature_2026010901 AS
WITH cus_sample AS (
    SELECT 
        mobile_md5,
        back_date
    FROM(
        -- 快递业务样本表
        select 
            phone as mobile_md5,
            substr(submitdate, 0, 10) as back_date
        from sms_bd_data.ods_yxx_spn_sms_detail_di
        where submitdate>='2025-08-20'
        limit 50000
    )smp
    group by mobile_md5, back_date
),
customer_dwb_express AS (
    select
        mobile_md5, 
        back_date, 
        sign, 
        event_type,
        event_time
    from(
        select
            cus_smp.mobile_md5,
            cus_smp.back_date,
            dwb_express.sign,
            dwb_express.event_type,
            dwb_express.event_time
        from cus_sample cus_smp
        left join(
            select
                phone,
                sign,
                event_type,
                from_unixtime(unix_timestamp(the_date, 'yyyyMMdd'), 'yyyy-MM-dd') as event_time
     

In [21]:
exp_field_mapping_dataframe = pd.DataFrame(exp_field_mapping_dict).T
exp_field_mapping_dataframe.to_excel("exp_field_mapping_output_v2.xlsx", index_label="field_name")

In [22]:
# 3. 输出字段类型统计
print("\n===== Express模块字段类型统计 =====")
type_count = {}
for field, meta in exp_field_mapping_dict.items():
    field_type = meta["type"]
    type_count[field_type] = type_count.get(field_type, 0) + 1
for t, cnt in type_count.items():
    print(f"{t}类型字段数：{cnt}")


===== Express模块字段类型统计 =====
STRING类型字段数：1
DATE类型字段数：1
INT类型字段数：33
BIGINT类型字段数：25
DOUBLE类型字段数：32
